# DICOM images resampling, 3D

This notebook is dedicated to the 3D resampling of the DICOM patient images.

Prerequisutes:
- Python 3.7.x, mine from Anaconda (5.3.1 x64 as of now, https://www.anaconda.com/download/)
- conda update python numpy matplotlib scipy pandas bokeh
- conda update pip
- pip install pydicom (v1.2 as of now)
- conda install plotly
- conda install scikit-image (shall be in full Anaconda, if miniConda is installed, you have to install it)

In [ ]:
%matplotlib inline

import numpy as np
import pydicom
from pathlib import Path
import matplotlib.pyplot as plt
import scipy.ndimage
from skimage import morphology
from skimage import measure
from skimage.transform import resize
import copy
import uuid

In [ ]:
X = 0
Y = 1
Z = 2

List directory and check and count files

In [ ]:
data_path = "D:/Patients/QT"
output_path = working_path = "."
output_dir  = "Out"

# here is new spacing to be resampled to
desired_spacing = [1.06445312, 1.06445312, 0.79]

# new patient name and ID, not to mess with existing ones,
# if None will be left unchanged
PatientName = "TEST^PAT" # = None
PatientID   = "7366473"  # = None

# new patient image orientation, if None will be left unchanged
theta = np.pi
phi   = 0.0
PatientImageOrientation = [np.cos(theta), np.sin(theta)*np.cos(phi), np.sin(theta)*np.sin(phi),
                           -np.cos(theta-0.5*np.pi), np.sin(theta-0.5*np.pi)*np.cos(phi+np.pi), np.sin(theta-0.5*np.pi)*np.sin(phi+np.pi)]
PatientImageOrientation = np.round(PatientImageOrientation, 7) # round to float precision
print(PatientImageOrientation)

l = list(Path(data_path).glob("*.dcm"))

# Print out the first 5 file names to verify we're in the right folder.
print("Total of {0} DICOM images.\nFirst 5 filenames:".format(len(l)))
for p in l[:5]:
    print(f"{p}")

Load all scans, set thickness

In [ ]:
def load_scans(path):
    """Here we assume equidistant in Z slices,
       sort output by InstanceNumber"""
    p = Path(path)
    slices = [pydicom.dcmread(str(Path(path) / s)) for s in p.iterdir() if s.is_file()]
    slices.sort(key = lambda x: int(x.InstanceNumber))
    try:
        slice_thickness = np.fabs(slices[0].ImagePositionPatient[2] - slices[1].ImagePositionPatient[2])
    except:
        slice_thickness = np.fabs(slices[0].SliceLocation - slices[1].SliceLocation)
        
    #for s in slices: # assign all slices the same thickness
    #    s.SliceThickness = slice_thickness
        
    return slices

id = 0
patient = load_scans(data_path)

# ok, put all images together into 3d matrix, test images dimensions as well
images  = np.stack([s.pixel_array for s in patient], axis = Z) # so final set of images would be (512,512,168)
print(f"All images together: {images.shape}")
print(patient[0]) # info from the first slice

In [ ]:
print(patient[167]) # last slice

Some good checks on what we got there

In [ ]:
sth = patient[0].SliceThickness
print(f"Slice Thickness: {sth}")
px0 = patient[0].PixelSpacing[X]
px1 = patient[0].PixelSpacing[Y]
print(f"Pixel Spacing (row, col): ({px0}, {px1})")

More checks - slopes, intercepts, slice positions etc

In [ ]:
for k, slice in enumerate(patient):
    minpix = np.min(slice.pixel_array.astype(np.int16))
    maxpix = np.max(slice.pixel_array.astype(np.int16))
    if minpix < 0:
        print(f"Negative value discovered: {k}")
        break
    if maxpix > 4095: # 12 bits scanner
        print(f"12-Positive value discovered:{k}")
        break
        
# now test for different pixel sizes
vx_prev = patient[0].PixelSpacing[X]
vy_prev = patient[0].PixelSpacing[Y]

for k, slice in enumerate(patient):
    vx = slice.PixelSpacing[X]
    vy = slice.PixelSpacing[Y]
    if np.fabs(vx - vx_prev) > 0.001:
        print(f"X pixel values are different:{k}")
        break
    if np.fabs(vy - vy_prev) > 0.001:
        print(f"Y pixel values are different:{k}")
        break
    vy_prev = vy
    vx_prev = vx
    
# now test for slope/intercept
slope_prev = patient[0].RescaleSlope
intrc_prev = patient[0].RescaleIntercept
for k, slice in enumerate(patient):
    slope = slice.RescaleSlope
    intrc = slice.RescaleIntercept
    if np.fabs(slope - slope_prev) > 0.001:
        print(f"Slope values are different:{k}")
        break
    if np.fabs(intrc - intrc_prev) > 0.001:
        print(f"Intercept values are different:{k}")
        break
    intrc_prev = intrc
    slope_prev = slope
    

# now test for position/thickness
patientpos_prev = None
sliceths_prev   = None
sliceloc_prev   = None
for k, slice in enumerate(patient):
    sliceths = slice.SliceThickness
    sliceloc = slice.SliceLocation
    
    if np.fabs( sliceloc - slice.ImagePositionPatient[Z] ) > 0.001:
        print(f"Slice position and patient position are different:{k}")
        break
        
    if k != 0:
        t1 = np.fabs(slice.ImagePositionPatient[Z] - patientpos_prev[Z]) # might be negative
        t2 = sliceths 
        t3 = np.fabs(sliceloc - sliceloc_prev) # might be negative
        
        if np.fabs(t1 - t2) > 0.001:
            print(f"Thickness from SliceThickness and PatientPosition are different:{k}")
        if np.fabs(t1 - t3) > 0.001:
            print(f"Thickness from SliceThickness and SliceLocation are different:{k}")
        
    patientpos_prev = slice.ImagePositionPatient
    sliceths_prev   = sliceths
    sliceloc_prev   = sliceloc

In [ ]:
np.save(str(Path(output_path)/f"fullimages_{id}.npy"), images)

In [ ]:
file_used = str(Path(output_path)/f"fullimages_{id}.npy")
imgs_to_process = np.load(file_used).astype(np.float64) 

plt.hist(imgs_to_process.flatten(), bins=50, color='c')
plt.xlabel("Voxel values")
plt.ylabel("Frequency")
plt.show()

In [ ]:
id = 0
imgs_to_process = np.load(file_used)

def sample_stack(stack, rows=9, cols=10, start_with=0, show_every=2):
    fig,ax = plt.subplots(rows, cols, figsize=[24, 24])
    last = False
    for ir in range(rows):
        for ic in range(cols):
            ind = start_with + (ir*cols + ic)*show_every
            if ind > stack.shape[Z] - 1:
                ind = stack.shape[Z] - 1
                last = True
            
            ax[ir, ic].set_title(f"slice {ind}")
            ax[ir, ic].imshow(stack[:,:,ind], cmap = "gray")
            ax[ir, ic].axis("off")
            if last:
                break
    plt.show()

sample_stack(imgs_to_process)

In [ ]:
id = 0
imgs_to_process = np.load(file_used)

def resample2d(image, scan, new_spacing):
    # Determine current pixel spacing
    spacing = map(float, [scan[0].PixelSpacing[X], scan[0].PixelSpacing[Y], scan[0].SliceThickness])
    spacing = np.array(list(spacing))

    resize_x    = spacing[X] / new_spacing[X]
    new_shape_x = np.round(image.shape[X] * resize_x)
    resize_x    = float(new_shape_x) / float(image.shape[X])
    sx = spacing[X] / resize_x

    resize_y    = spacing[Y] / new_spacing[Y]
    new_shape_y = np.round(image.shape[Y] * resize_y)
    resize_y    = new_shape_y / image.shape[Y]
    sy = spacing[Y] / resize_y
    
    image = scipy.ndimage.interpolation.zoom(image, (resize_x, resize_y, 1.0), order=1)
    
    return (image, (sx, sy))

def resample3d(image, scan, new_spacing):
    # Determine current pixel spacing
    spacing = map(float, [scan[0].PixelSpacing[X], scan[0].PixelSpacing[Y], scan[0].SliceThickness])
    spacing = np.array(list(spacing))
    
    resize_x    = spacing[X] / new_spacing[X]
    new_shape_x = np.round(image.shape[X] * resize_x)
    resize_x    = float(new_shape_x) / float(image.shape[X])
    sx = spacing[X] / resize_x

    resize_y    = spacing[Y] / new_spacing[Y]
    new_shape_y = np.round(image.shape[Y] * resize_y)
    resize_y    = new_shape_y / image.shape[Y]
    sy = spacing[Y] / resize_y

    resize_z    = spacing[Z] / new_spacing[Z]
    new_shape_z = np.round(image.shape[Z] * resize_z)
    resize_z    = float(new_shape_z) / float(image.shape[Z])
    sz = spacing[Z] / resize_z
    
    image = scipy.ndimage.interpolation.zoom(image, (resize_x, resize_y, resize_z), order=1)
    
    return (image, (sx, sy, sz))

print(f"Shape before resampling\t{imgs_to_process.shape}")
imgs_after_resamp, spacing = resample3d(imgs_to_process, patient, desired_spacing)
print(f"Shape after resampling:\t{imgs_after_resamp.shape}")
print(f"New spacing: {spacing}")

In [ ]:
p = Path(output_dir)
if not p.is_dir():
    p.mkdir(parents = True, exist_ok = True)    

l  = imgs_after_resamp.shape[Z]

zbeg = patient[0].ImagePositionPatient[Z]
zsgn = 1.0 if patient[0].ImagePositionPatient[Z] < patient[1].ImagePositionPatient[Z] else -1.0
zthk = np.round(spacing[Z], decimals=2)

# print(f"{zbeg} {zsgn} {zthk}")

if l == len(patient): # same number of slices
    for k, slice in enumerate(patient):
        q = imgs_after_resamp[:, :, k]
        slice.PixelData = q.tobytes()
        slice.PixelSpacing[X] = np.round(spacing[X], decimals=2)
        slice.PixelSpacing[Y] = np.round(spacing[Y], decimals=2)
        slice.Rows, slice.Columns = q.shape
        slice.PatientName = slice.PatientName if PatientName is None else PatientName
        slice.PatientID   = slice.PatientID if PatientID is None else PatientID
        if PatientImageOrientation is not None:
            slice.ImageOrientationPatient[0] = PatientImageOrientation[0]
            slice.ImageOrientationPatient[1] = PatientImageOrientation[1]
            slice.ImageOrientationPatient[2] = PatientImageOrientation[2]
            slice.ImageOrientationPatient[3] = PatientImageOrientation[3]
            slice.ImageOrientationPatient[4] = PatientImageOrientation[4]
            slice.ImageOrientationPatient[5] = PatientImageOrientation[5]
            
        #slice.SliceThickness          = zthk
        
        #loc = zbeg + float(k) * zsgn * zthk
        #slice.ImagePositionPatient[Z] = loc
        #slice.SliceLocation           = loc                

        fname = Path(output_dir)/(slice.SeriesInstanceUID + "_" + str(slice.InstanceNumber) + ".dcm")
        pydicom.dcmwrite(str(fname), slice, False)
        
else: # we added/removed slices
    rc = list()
    for k in range(0, l):
        slice = None
        if k < len(patient):
            slice = copy.deepcopy(patient[k])
        else:
            slice = copy.deepcopy(patient[-1])
            
            pos = slice.SOPInstanceUID.rfind(".")
            uid = str(int(uuid.uuid4().hex, 16))
            slice.SOPInstanceUID = slice.SOPInstanceUID[:pos+1] + uid
        
        q = imgs_after_resamp[:, :, k]
        slice.PixelData = q.tobytes()
        slice.PixelSpacing[X] = np.round(spacing[X], decimals=2)
        slice.PixelSpacing[Y] = np.round(spacing[Y], decimals=2)
        slice.Rows, slice.Columns = q.shape
        slice.PatientName = slice.PatientName if PatientName is None else PatientName
        slice.PatientID   = slice.PatientID if PatientID is None else PatientID
        if PatientImageOrientation is not None:
            slice.ImageOrientationPatient[0] = PatientImageOrientation[0]
            slice.ImageOrientationPatient[1] = PatientImageOrientation[1]
            slice.ImageOrientationPatient[2] = PatientImageOrientation[2]
            slice.ImageOrientationPatient[3] = PatientImageOrientation[3]
            slice.ImageOrientationPatient[4] = PatientImageOrientation[4]
            slice.ImageOrientationPatient[5] = PatientImageOrientation[5]
        
        slice.SliceThickness          = zthk
        
        loc = zbeg + float(k) * zsgn * zthk
        slice.ImagePositionPatient[Z] = loc
        slice.SliceLocation           = loc
        
        slice.InstanceNumber = k + 1
        
        fname = Path(output_dir)/(slice.SeriesInstanceUID + "_" + str(slice.InstanceNumber) + ".dcm")
        pydicom.dcmwrite(str(fname), slice, False)
        
print(f"Done! {l} slices were written")

Check output directory for images

### References

Inspired by https://www.raddq.com/dicom-processing-segmentation-visualization-in-python/